In [246]:
import re
from itertools import groupby
from datetime import datetime

In [247]:
def returnValue(val):
    return val.group(1) if  val else None

In [248]:
def isPredictMsg(msg):

    patterns = [
        r"Symbol: (.+)",
        r"Position: (.+)",
        r"Leverage: (.+)",
        r"Market: (.+)",
        r"StopLoss: (.+)"
    ]

    # Check if all patterns have a value
    return all(re.search(pattern, msg) for pattern in patterns)

In [249]:
def findPosition(msg):
    if msg.find("📈") != -1:
        return "LONG"
    elif msg.find("📉") != -1:
        return "SHORT"
    return None
    
    

In [250]:
msg = {
    "id": 8445,
    "date": "2023-12-01T15:25:59+00:00",
    "reply_to_msg_id": None,
    "message": "🔥 #CHR/USDT (Short📉, x20) 🔥\n\nEntry - 0.2312\nTake-Profit:\n\n 0.2267 (40% of profit)\n🥈 0.2245 (60% of profit)\n🥇 3.2724 (80% of profit)\n🚀 3.3068 (100% of profit)\n@DigiLe@persianeliteorg",
    "edit_date": None,
    "media": None,
}

string = msg["message"]

# print(string)
symbol_match = string[string.find("#") + 1 : string.find("/USDT")].strip() + "-USDT"
position_match = findPosition(string)
leverage_match = re.search(r"x(\d+)", string)
market_match = "FUTURES"
entry_match = re.search(r"Entry - (.+)", string)
stopLoss_match = float(returnValue(entry_match)) * 0.9


profit_values = re.findall(r"([\d.]+)\s*\(", string)


# Creating a dictionary
data = {
    "symbol": symbol_match,
    "position": position_match,
    "market": market_match,
    "leverage": returnValue(leverage_match),
    "stopLoss": stopLoss_match,
    "entry Targets": {"Target 1": returnValue(entry_match)},
    "take-Profit Targets": {
        f"Target {i+1}": value for i, value in enumerate(profit_values)
    }
    if profit_values
    else None,
}

data
# print(data)

{'symbol': 'LDO-USDT',
 'position': 'LONG',
 'market': 'FUTURES',
 'leverage': '20',
 'stopLoss': 2.82735,
 'entry Targets': {'Target 1': '3.1415'},
 'take-Profit Targets': {'Target 1': '3.2056',
  'Target 2': '3.2387',
  'Target 3': '3.2724',
  'Target 4': '3.3068'}}

In [251]:
def isEntry(msg, value, symbol):
    entry_price = returnValue(re.search(r"Entry Price: (.+)", msg))
    # for control "average entry"
    if entry_price:
        entry_price = float(re.findall(r"\d+\.\d+", entry_price)[0])
        bigger_number = max(entry_price, float(value))
        smaller_number = min(entry_price, float(value))

        error = 100 * (1 - (smaller_number / bigger_number)) > 1
        if error:
            return False
    else:
        return False

    patterns = [
        r"Entry(.+)",
        rf"{symbol}",

    ]

    # Check if all patterns have a value

    return all(re.search(pattern, msg) for pattern in patterns)

In [252]:
def isTakeProfit(msg, symbol, index):
   

    patterns = [
        r"Take-Profit(.+)",
        r"Profit(.+)",
        rf"target {index}",
        rf"{symbol}",
    ]

    # Check if all patterns have a value
    return all(re.search(pattern, msg) for pattern in patterns)

In [253]:
input_str = "🔥 #LDO/USDT (Long📈, x20) 🔥\n\nEntry - 3.1415\nTake-Profit:\n\n🥉 3.2056 (40% of profit)\n🥈 3.2387 (60% of profit)\n🥇 3.2724 (80% of profit)\n🚀 3.3068 (100% of profit)\n@DigiLe@persianeliteorg"

# Find all float numbers before "("
floats_before_parentheses = re.findall(r"([\d.]+)\s*\(", input_str)

# Find all float numbers after "Take-Profit:"
floats_after_take_profit = re.findall(r"Take-Profit:\s*([\d.]+)", input_str)

print("Floats before '(': ", floats_before_parentheses)
print("Floats after 'Take-Profit:': ", floats_after_take_profit)

Floats before '(':  ['3.2056', '3.2387', '3.2724', '3.3068']
Floats after 'Take-Profit:':  []


In [81]:
def sizeAmount(price):
    prices = [
        { 'price':0.01, 'size':2000 },
        { 'price':0.1, 'size':200 },
        { 'price':1, 'size':20 },
        { 'price':5, 'size':2 },
        ]
    
    length = len(prices) - 1
    for i in range(len(prices)):
        if i == length: return prices[0]["size"]
        if prices[i]["price"] <= price < prices[i+1]["price"]:
            size = prices[i+1]["size"]
            return size if size is not None else prices[i]["size"]
        


200
